In [ ]:
%pip install -U -q google-genai gradio

# Sumarización con Google Gemini API y Gradio
Esta interfaz te permite sumarizar textos utilizando el modelo Gemini de Google.

In [ ]:
from google.colab import userdata
import os

GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')

In [ ]:
from google import genai
from google.genai import types
import gradio as gr

cliente = genai.Client(api_key=GOOGLE_API_KEY)

In [ ]:
MODEL_ID = "gemini-2.0-flash-lite" # @param ["gemini-2.0-flash-lite","gemini-2.0-flash","gemini-2.5-flash-preview-05-20","gemini-2.5-pro-preview-05-06"] {"allow-input":true, isTemplate: true}

# Texto de ejemplo

In [ ]:
# Texto de ejemplo que se cargará por defecto
texto_ejemplo = """Un hombre fue detenido por la PFA por una serie de amenazas de muerte dirigidas a Patricia Bullrich a través de redes sociales. En el marco de una investigación coordinada por el Ministerio de Seguridad Nacional, efectivos del Departamento Federal de Investigaciones arrestó, en las últimas horas, al supuesto responsable en la Ciudad de Buenos Aires. La causa se inició el 20 de marzo tras una denuncia presentada por el Ministerio, luego de que la funcionaria del gobierno de Javier Milei, recibiera las intimidaciones. Entre los mensajes se encontraban expresiones como: “¿No te duele ser tan fascista? Personas como vos merecen morir antes que contratemos un sicario por la Deep Web para mandarte a hacer acá, querida basurita” y “Ya te voy a matar a vos, algún día vas a entrar al Congreso, hija de puta. Antes de que vuelvas a ajustar y a cagar a palos a los jubilados como en el 2001”. Ante la gravedad de los hechos, intervino la Unidad de Investigación Antiterrorista de la PFA, que logró identificar el domicilio del autor de los mensajes, identificado como Leandro Gabriel Gaona, ubicado en la calle Luzuriaga, detallaron fuentes el caso a Infobae. Con esa información, el Juzgado Nacional en lo Criminal y Correccional Federal N°12, a cargo de Ariel Lijo y la Secretaría N°23 de Paola Yanina Kohen, ordenó el allanamiento del inmueble. Durante el operativo, en el que participaron efectivos del GEOF, la División Canes y la Brigada de Explosivos, fue detenido Gaona, un ciudadano argentino de 37 años. Además, en el lugar se secuestró un artefacto explosivo casero tipo “Molotov”, un teléfono celular, un CPU, bibliografía con contenido anarquista, panfletos y material con consignas antisistémicas, indicaron. Noticia en desarrollo"""

## Función de Sumarización

In [ ]:
def sumarizar_texto(texto, tipo_resumen):
    """Función que sumariza el texto usando Gemini"""

    if not texto.strip():
        return "Por favor, ingresa un texto para sumarizar."

    # Definir diferentes tipos de prompt según la opción seleccionada
    prompts = {
        "Rápido (3 oraciones)": f"Sumariza el siguiente texto en tres oraciones de rápida lectura:\n\nTexto: {texto}",
        "Detallado (6 líneas)": f"Resumí el siguiente texto en no más de 6 líneas:\n\nTexto: {texto}",
        "Muy breve (1 oración)": f"Resumí el siguiente texto en UNA sola oración:\n\nTexto: {texto}"
    }

    try:
        respuesta = cliente.models.generate_content(
            model=MODEL_ID,
            contents=[prompts[tipo_resumen]]
        )
        return respuesta.text

    except Exception as e:
        return f"Error al procesar el texto: {str(e)}"

## Interface Gradio

In [ ]:
# Crear la interfaz Gradio
interface = gr.Interface(
    fn=sumarizar_texto,
    inputs=[
        gr.Textbox(
            label="Texto a sumarizar",
            placeholder="Pega acá el texto que queres sumarizar...",
            lines=8,
            value=texto_ejemplo
        ),
        gr.Dropdown(
            choices=["Rápido (3 oraciones)", "Detallado (6 líneas)", "Muy breve (1 oración)"],
            label="Tipo de resumen",
            value="Rápido (2 oraciones)"
        )
    ],
    outputs=gr.Textbox(
        label="Resumen generado",
        lines=4
    ),
    title="Sumarizador con Gemini",
    description="Ingresa cualquier texto y elegí el tipo de resumen que preferis. El modelo Gemini generará un resumen automáticamente.",
    examples=[
        ["Estimado Amazon, la semana pasada pedí una figura de acción de Optimus Prime en su tienda en línea en Alemania.", "Rápido (2 oraciones)"],
    ],
    theme=gr.themes.Soft()
)

# Lanzar la interfaz
interface.launch()